In [1]:
import os
import numpy as np
import pandas as pd

import geopandas as gpd
import rioxarray as rioxr
from random import sample

from shapely.geometry import Point 

from rasterio.crs import CRS

In [3]:
# Load raster
year = 2020

root = '/home/jovyan/msai4earth-esa/iceplant_detection/processing_results/' + os.path.join('LS_filter_clip_preds_' + str(year))

fp = os.path.join(root, 'LS_merged_crs26910_S_'+str(year)+'.tif')
r_26910_S = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, 'LS_merged_crs26910_W_'+str(year)+'.tif')
r_26910_W = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, 'LS_merged_crs26911_'+str(year)+'.tif')
r_26911 = rioxr.open_rasterio(fp).squeeze()

rasters = [r_26910_S, r_26910_W, r_26911]

In [4]:
pixel_count = pd.read_csv(os.path.join(os.getcwd(), 'rasters_2020_pixel_counts.csv'))
pixel_count

,n_nonice_20,n_ice_20,n_ground_20,n_water_20,raster
0,32334429,9319051,111150412,62968690,LS_merged_crs26910_S_2020
1,887994,264213,1891593,2893071,LS_merged_crs26910_W_2020
2,87881212,2912345,62587031,69125241,LS_merged_crs26911_2020_clip


In [6]:
# STRATIFIED RANDOM SAMPLING
cats = [0, 1, 2, 3]
samples_per_class = [100, 100, 150, 115]
dist = []

for c in cats:
    
    # pixels in each raster from specified class
    n1 = list(pixel_count.iloc[:,c])[0]
    n2 = list(pixel_count.iloc[:,c])[1]
    n3 = list(pixel_count.iloc[:,c])[2]
    s = n1 + n2 + n3
     
    # randomly specify number of samples per raster
    where_to_sample = np.random.choice([0,1,2], size=samples_per_class[c], p=[n1/s, n2/s, n3/s])

    dist.append([len(np.where(where_to_sample == 0)[0]), 
                 len(np.where(where_to_sample == 1)[0]), 
                 len(np.where(where_to_sample == 2)[0])])
    
dist

[[24, 0, 76], [73, 3, 24], [90, 1, 59], [49, 0, 66]]

In [ ]:
all_points = []

for i in range(0,3):

    points = []
    which_class = []
    which_raster = []
    sampled_raster = False
    print(' STARTED COLLECTING FOR RASTER ', i)
    
    for c in cats:
        n_samples = dist[c][i]

        if n_samples != 0:
            sampled_raster = True

            raster = rasters[i]

            # arrays with indices where condition is True
            c_raster = np.where(raster == c)

            # sample random (y,x) pairs from list
            indices = sample(list(zip(c_raster[0], c_raster[1])), n_samples)

            # unzip into y and x coordinates
            y, x = list(zip(*indices))

            # x and y coordinates in raster CRS corresponding to sampled points
            x_coord = raster.x[np.asarray(x)]
            y_coord = raster.y[np.asarray(y)]

            # make points 
            points = points + [Point(m,n) for m,n in zip(x_coord, y_coord)]

            which_class = which_class + [c]*n_samples
            which_raster = which_raster + [i]*n_samples
            print('sampled cat ', c)
            
    if sampled_raster:
        points_df = gpd.GeoDataFrame({'geometry': points,
                                      'class' : which_class,
                                      'which_raster' : which_raster},
                                     crs = raster.rio.crs)
        all_points.append(points_df.to_crs(CRS.from_epsg(4326)))  # change to lat/lon crs
    print(' ')
        

 STARTED COLLECTING FOR RASTER  0
sampled cat  0
sampled cat  1
sampled cat  2
sampled cat  3
 
 STARTED COLLECTING FOR RASTER  1
sampled cat  1
sampled cat  2
 
 STARTED COLLECTING FOR RASTER  2
sampled cat  0
sampled cat  1


In [ ]:
points_df = pd.concat(all_points, ignore_index=True)
points_df['LON'] = points_df.geometry.x
points_df['LAT'] = points_df.geometry.y
points_df = points_df.drop(['geometry'], axis=1)
points_df

In [7]:
points_df.to_csv('validation_sample_2020.csv', index_label='PLOTID')

In [12]:
np.unique(points_df['class'], return_counts=True)

(array([0, 1, 2, 3]), array([ 87,  87,  91, 135]))

In [13]:
np.unique(points_df['which_raster'], return_counts=True)

(array([0, 1, 2]), array([210,  11, 179]))